In [1]:
# Step 1: Install dependencies
!pip install rembg opencv-python pillow onnxruntime

import cv2
import numpy as np
from PIL import Image
from rembg import remove
from google.colab import files
import io

In [45]:
# Step 2: Upload photo
uploaded = files.upload()
input_path = list(uploaded.keys())[0]

Saving IMG_20250921_131245.jpg to IMG_20250921_131245.jpg


In [59]:
# Step 3: Background removal + passport formatting
def process_passport_photo(input_path, output_path="passport_photo.jpg", bg_color="white", target_size=(1200, 1200)):
    try:
        # Open image
        input_img = Image.open(input_path)

        # Remove background
        output = remove(input_img)

        # Convert to numpy array (with alpha)
        output_np = np.array(output).copy()

        # Transparent background handling
        if bg_color == "transparent":
            final_img = Image.fromarray(output_np, "RGBA")
            output_path = output_path.replace('.jpg', '.png')  # Save as PNG to preserve transparency
            final_img.save(output_path)
            return final_img, output_path

        # If solid background requested
        if bg_color in ["white", "blue"]:
            h, w, _ = output_np.shape
            if bg_color == "white":
                bg = np.ones((h, w, 3), dtype=np.uint8) * 255
            elif bg_color == "blue":
                bg = np.array([0, 0, 255], dtype=np.uint8)  # Blue background
                bg = np.tile(bg, (h, w, 1))

            # Blend foreground with background using alpha channel
            alpha = output_np[:, :, 3] / 255.0
            for c in range(3):
                output_np[:, :, c] = (alpha * output_np[:, :, c] +
                                      (1 - alpha) * bg[:, :, c])

        # Convert back to PIL, convert to RGB since background is solid
        final_img = Image.fromarray(output_np).convert("RGB")

        # Helper function to resize with padding
        def resize_with_padding(img, target_size=target_size, bg_color=(255, 255, 255)):
            old_size = img.size  # (width, height)

            # Calculate ratio to keep aspect ratio
            ratio = min(target_size[0] / old_size[0], target_size[1] / old_size[1])
            new_size = (int(old_size[0] * ratio), int(old_size[1] * ratio))

            # Resize with LANCZOS resampling for quality
            img = img.resize(new_size, Image.LANCZOS)

            # Create background and paste resized image at center
            new_img = Image.new("RGB", target_size, bg_color)
            new_img.paste(img, ((target_size[0] - new_size[0]) // 2,
                                (target_size[1] - new_size[1]) // 2))
            return new_img

        # Resize with padding using selected bg_color
        if bg_color == "blue":
            final_img = resize_with_padding(final_img, target_size, bg_color=(0, 0, 255))
        else:
            final_img = resize_with_padding(final_img, target_size, bg_color=(255, 255, 255))

        # Save final image with high quality settings
        if output_path.lower().endswith(('.jpg', '.jpeg')):
            final_img.save(output_path, quality=100, dpi=(300, 300))
        else:
            final_img.save(output_path, dpi=(300, 300))

        return final_img, output_path

    except Exception as e:
        print(f"Error processing image: {e}")
        return None, None

In [60]:
# Step 4: Run processing (change bg_color = "white"/"blue"/"transparent")
final_img, output_file = process_passport_photo(input_path, bg_color="white")

In [61]:
# Step 5: Show output
final_img.show()

In [62]:
# Step 6: Download processed passport photo
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>